In [ ]:
!pip install scikit-learn
!pip install torch
!pip install transformers
!pip install datasets

In [3]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
import torch.nn.functional as F
import matplotlib.pyplot as plt
from datasets import load_dataset

In [ ]:
def get_embedding_vector_for_string(prompt, model, tokenizer):
    # Tokenize the input string and move to the model's device
    seq_ids = tokenizer(prompt, return_tensors='pt', truncation=True, padding=True).to(model.device)["input_ids"]

    # Compute the embeddings
    with torch.no_grad():
        output = model(seq_ids)
        embedding = output["last_hidden_state"].mean(dim=1).squeeze()

    del seq_ids, output
    torch.cuda.empty_cache()
    gc.collect()

    return embedding  # Return detached embedding

def extract_openai_data(example):
    prompt = example["info"]["post"]
    summaries = example["summaries"]
    chosen_idx = example["choice"]

    prompt = get_embedding_vector_for_string(prompt, model, tokenizer)
    positive = get_embedding_vector_for_string(summaries[chosen_idx]["text"], model, tokenizer)
    negative = get_embedding_vector_for_string(summaries[1 - chosen_idx]["text"], model, tokenizer)
    user_id = example["worker"]

    return {"prompt": prompt, "positive": positive, "negative": negative, "user_id": user_id}

def extract_chatbotarena_data(example):
    winner = example["winner"]
    loser = "conversation_b" if winner == "conversation_a" else "conversation_a"

    prompt = get_embedding_vector_for_string(example[winner][0]["content"], model, tokenizer)
    positive = get_embedding_vector_for_string(example[winner][1]["content"], model, tokenizer)
    negative = get_embedding_vector_for_string(example[loser][1]["content"], model, tokenizer)
    user_id = example["judge"]

    return {"prompt": prompt, "positive": positive, "negative": negative, "user_id": user_id}

def retrieve_info_from_data(data, with_prompt: bool=False):
    if with_prompt:
        positives = torch.stack([torch.cat((item["positive"], item["prompt"]), dim=1) for item in data])
        negatives = torch.stack([torch.cat((item["negative"], item["prompt"]), dim=1) for item in data])
    else:
        positives = torch.stack([item["positive"] for item in data])
        negatives = torch.stack([item["negative"] for item in data])

    datapoints = positives - negatives
    return datapoints

In [ ]:
# Global constants, TODO: edit based on model architecture/dataset/task
metric_space_dim = 2
embedding_dim = 2
input_dim = 2
hidden_dim = 4
K = 3

model_name = "meta-llama/Llama-3.2-1b-hf"
# model_name = "google/gemma-2-2b"
batch_size = 16

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

In [ ]:
# Mapping version.

import gc

# Use OpenAI dataset
dataset = load_dataset("openai/summarize_from_feedback", "comparisons")
print("Dataset loaded.")

# Use Chatbot Arena dataset
# dataset = load_dataset("chatbot_arena_conversation")
# extracted_data = dataset.map(extract_chatbotarena_data)

subset = dataset["train"].select(range(1000))
del dataset
extracted_data = subset.map(extract_openai_data)
print("Dataset extracted.")

del model
del tokenizer
del dataset

gc.collect()

X = retrieve_info_from_data(extracted_data)
print("Dataset retrieved.")

In [ ]:
# Iterative lazy version.

import itertools
import gc

# Use OpenAI dataset
dataset = load_dataset("openai/summarize_from_feedback", "comparisons", streaming=True)
print("Dataset loaded.")

subset = itertools.islice(dataset["train"], 1000)
print("Dataset subset loaded.")

batch_size = 16
extracted_data = []
batch = []
examples = 0

for example in subset:
    examples += 1
    batch.append(example)

    if len(batch) == batch_size:
        extracted_data.extend(batch.map(extract_openai_data))
        batch = []  # Reset batch

    if examples % 64 == 0:
        print(f"Examples processed: {examples}")

del model
del tokenizer
del dataset

gc.collect()

X = retrieve_info_from_data(extracted_data)
print("Dataset retrieved.")

In [ ]:
N = 100
silhouette_scores = []

for n in range(2, N + 1):
    print(f"Clustering with k={n} calculating.")
    kmeans = KMeans(n_clusters=n, n_init="auto").fit(X.cpu())
    score = silhouette_score(X.cpu(), kmeans.fit_predict(X.cpu()))
    silhouette_scores.append(score)
    print(f"Silhouette score is {score}.")

plt.figure(figsize=(8, 5))
plt.plot(range(2, N + 1), silhouette_scores, marker='o')
plt.title("Silhouette Score vs. Number of Clusters")
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Score")
plt.grid()
plt.show()